In [46]:
import numpy as np
import pprint
import copy
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [71]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()


In [72]:
# Taken from Policy Evaluation Exercise!

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)

In [73]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    step = 0
    while True:
        # Implement this!
        end_flug = True
        old_policy = copy.deepcopy(policy)
        v_policy = policy_eval_fn(policy,env)
        step = step+1
        #Policy Improvement 
        print("---------------------policy improvement. step:{}------------------------------------".format(step))
        print(policy)
        for s in range(env.nS):
            greedy_action = 0
            max_q = sys.float_info.min
            q = np.zeros(env.nA)
            for a in range(env.nA):
                for  prob, next_state, reward, done in env.P[s][a]:
                    q[a] += reward + discount_factor * v_policy[next_state]
            greedy_action = np.argmax(q)
            print("s:{}, q_value:{}, argmax of q:{}".format(s,q,greedy_action))
            policy[s] = np.zeros(env.nA)
            policy[s][greedy_action] = 1.0
            print(old_policy[s])
            print(policy[s])
            if (old_policy[s] != policy[s]).all():
                print("not match! in s:{}".format(s))
                end_flug = False
        if end_flug:
            
            print("---------------------policy improvement finish !------------------------------------")
            break
        
    
    return policy, policy_eval_fn(policy,env)

In [74]:
policy, v = policy_improvement(env,policy_eval)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



---------------------policy improvement. step:1------------------------------------
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
s:0, q_value:[0. 0. 0. 0.], argmax of q:0
[0.25 0.25 0.25 0.25]
[1. 0. 0. 0.]
not match! in s:0
s:1, q_value:[-14.99993529 -20.99990698 -18.9999206   -1.        ], argmax of q:3
[0.25 0.25 0.25 0.25]
[0. 0. 0. 1.]
not match! in s:1
s:2, q_value:[-20.99990698 -22.99989761 -20.99991379 -14.99993529], argmax of q:3
[0.25 0.25 0.25 0.25]
[0. 0. 0. 1.]
not match! in s:2
s:3, q_value:[-22.99989761 -22.99989761 -20.99991477 -20.99990698], argmax of q:3
[0.25 0.25 0.25 0.25]
[0. 0. 0. 1.]
not match! in s:3
s:4, q_value:[ -1.         -18.9999206  -20

In [75]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)